In [ ]:
# check math in forward_A model

import torch
from torch.nn import functional as F
from numpy import pi, exp, sqrt
import numpy as np
import matplotlib.pyplot as plt


def get_gaussian(side_len=5, s=1):
    k= (side_len-1)//2
    probs = [exp(-z*z/(2*s*s))/sqrt(2*pi*s*s) for z in range(-k,k+1)] 
    kernel = np.outer(probs, probs)
    return kernel

X= torch.randn((10, 1, 32, 32))#torch.ones((10, 1, 32, 32)) 
Ht = torch.randn((1, 1, 32, 32))#torch.zeros((1, 1, 32, 32)) 
sPSF = torch.tensor(get_gaussian(side_len=5, s=1)).float().view(1,1,5,5)
exPSF=torch.tensor(get_gaussian(side_len=5, s=1)).float().view(1,1,5,5)


#X: (m, 1, Nx, Ny), Ht: (1, 1, Nx, Ny), sPSF: (N1, N1), exPSF: (N2, N2)

A1= F.conv2d(Ht, exPSF, padding= 2)

plt.figure()
plt.subplot(1,3,1)
plt.imshow(Ht[0,0].detach().numpy(), cmap='gray', vmin=0, vmax=1)
plt.title('Ht')
plt.subplot(1,3,2)
plt.imshow(exPSF[0,0].detach().numpy())
plt.title('exPSF')
plt.subplot(1,3,3)
plt.imshow(A1[0,0].detach().numpy(), cmap='gray', vmin=0, vmax=1)
plt.title('A1')
plt.suptitle(f'Ht : {Ht.shape}, exPSF : {exPSF.shape}, A1(OUT) : {A1.shape}')
plt.show()


A2= A1*X

plt.figure()
plt.subplot(1,3,1)
plt.imshow(A1[0,0].detach().numpy(), cmap='gray', vmin=0, vmax=1)
plt.title('A1')
plt.subplot(1,3,2)
plt.imshow(X[0,0].detach().numpy(), cmap='gray', vmin=0, vmax=1)
plt.title('X')
plt.subplot(1,3,3)
plt.imshow(A2[0,0].detach().numpy(), cmap='gray', vmin=0, vmax=1)
plt.title('A2')
plt.suptitle(f'A1 {A1.shape}, X : {X.shape}, A2(OUT) {A2.shape}')
plt.show()

yt= F.conv2d(A2, sPSF, padding= 2)

plt.figure()
plt.subplot(1,3,1)
plt.imshow(A2[0,0].detach().numpy(), cmap='gray', vmin=0, vmax=1)
plt.title('A2')
plt.subplot(1,3,2)
plt.imshow(sPSF[0,0].detach().numpy())
plt.title('sPSF')
plt.subplot(1,3,3)
plt.imshow(yt[0,0].detach().numpy(), cmap='gray', vmin=0, vmax=1)
plt.title('yt')
plt.suptitle(f'A2 {A2.shape}, sPSF : {sPSF.shape}, yt(OUT) {yt.shape}')
plt.show()


In [ ]:
print(A1.shape, X.shape)
print((A1*X == A1.repeat(10, 1, 1, 1)*X).sum(), X.shape, 'they are same')
print(A1.repeat(10, 1, 1, 1).shape)

## Forward model- value check

In [1]:
import torch
from modules.models.forward_model import forward_modelA
from modules.kernels import get_gaussian


device = 'cuda'
X= torch.randint(0, 256, (32, 1, 32, 32))/255 #make positive
H = torch.randint(45, 2402, (1, 1, 32, 32))/1000000
sPSF = torch.tensor(get_gaussian(side_len=5, s=1)).float()
exPSF=torch.tensor(get_gaussian(side_len=5, s=1)).float()

yt = forward_modelA(X, H, sPSF, exPSF, device, noise=True)
print(f'yt range : {yt.min()}, {yt.max()}')

range(H): [4.70000013592653e-05, 0.002400000113993883] || range(X): [0.0, 1.0]
sample from normal : input(lambda) range : [5.48548050574027e-05, 0.0010308248456567526]
sample from normal : output(sample) range : [-0.1158958375453949, 0.1288934201002121]
yt range : -0.1158958375453949, 0.1288934201002121
